In this workbook we present the scripts to fill the database for visualization. We provide the following steps:

1.) Import data from the API
2.) Calculate the (relative) changes based on step 1
3.) Calculate the (conditional) Value at Risk
4.) Calculate teh correlation

In [ ]:
# import required packages
from app.lib import utils
import datetime
import pandas as pd
from itertools import compress, product

# set parameter
symbols = ["BTC-USD", "AAPL", "BBBY", "^GSPC", "GME", "^GDAXI", "EURUSD=X"]
start_date = "2015-11-17"
final_end_date = "2022-12-14"
interval = "1d"

In [ ]:
# get data from api

# https://stackoverflow.com/questions/26055556/writing-python-pandas-data-frame-to-sql-database-error
data = {}
for symbol in symbols:
    print(symbol)
    df = utils.extract_until_end_date(symbol, utils.str_to_datetime(start_date,format = '%Y-%m-%d')
    , utils.str_to_datetime(final_end_date,format = '%Y-%m-%d'), interval = interval, break_first_iteration = False)
    df["symbol"] = symbol
    df["interval"] = interval
    eng = utils.create_engine_object("SQLSERVER", my_host = "localhost", my_db = "master", my_odbc_driver = "ODBC Driver 17 for SQL Server")
    rc = utils.write_to_db(eng, df.copy(), "CANDLE_DATA")


In [ ]:
# calculate the change of previous data

if interval == "1m":
    start_date = datetime.datetime(2022, 11, 8)
    end_date = datetime.datetime(2022, 12, 3)
    dates = pd.date_range(start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d"), freq = "D")
    length_interval_in_daays = 1
elif interval == "1d":
    dates = [datetime.datetime(2000, 1, 1)]
    length_interval_in_daays = 365*100

df = {}
eng = utils.create_engine_object("SQLSERVER", my_host = "localhost", my_db = "master", my_odbc_driver = "ODBC Driver 17 for SQL Server")

for symbol in symbols:
    print(symbol)
    print(dates)
    df["__".join(symbol)] = utils.calculate_change(dates, symbol, eng, interval, length_interval_in_daays)


In [ ]:
# calculate var

start_date = datetime.datetime(2021, 12, 31)
end_date = datetime.datetime(2022, 12, 15)
period_length_in_days = [180, 365, 2*365, 5*365, 10 *365]

symbol_cobinations = ( set(compress(symbols,mask)) for mask in product(*[[0,1]]*len(symbols)))

# it is important to sort to always have same order!!
symbol_cobinations = [sorted(list(e)) for e in list(symbol_cobinations) if len(list(e))>1]

df = {}
eng = utils.create_engine_object("SQLSERVER", my_host = "localhost", my_db = "master", my_odbc_driver = "ODBC Driver 17 for SQL Server")

alpha=[0.1, 0.05, 0.01]
for symbols in symbol_cobinations:
    for p in period_length_in_days:
        d1 = start_date
        while d1<=end_date:
            print(symbols)
            utils.calculate_var(d1, symbols, eng, interval, alpha, p)
            # we calculate the VaR for each month
            d1 = d1  + pd.DateOffset(months=1)
            break
        break


In [ ]:
# calculate correlation

if interval == "1m":
    start_date = datetime.datetime(2022, 11, 17)
    end_date = datetime.datetime(2022, 12, 15)
    period_length_in_days = [1, 2, 5]
    days_offset = 1
elif interval == "1d":
    start_date = datetime.datetime(2000, 11, 17)
    end_date = datetime.datetime(2022, 12, 15)
    period_length_in_days = [30, 90, 100]
    days_offset = 10


symbol_cobinations = ( set(compress(symbols,mask)) for mask in product(*[[0,1]]*len(symbols)) )
# it is important to sort to always have same order!!
symbol_cobinations = [sorted(list(e)) for e in list(symbol_cobinations) if len(list(e))==2]

df = {}
eng = utils.create_engine_object("SQLSERVER", my_host = "localhost", my_db = "master", my_odbc_driver = "ODBC Driver 17 for SQL Server")

for symbols in symbol_cobinations:
    print(symbols)
    for p in period_length_in_days:
        d1 = start_date
        while d1<=end_date:
            utils.calculate_correlation(d1, symbols, eng, interval, p)
            d1 = d1  + pd.DateOffset(days=days_offset)